# Lab 5: Brightness and Contrast Adjustment

**Computer Vision Course**

In this lab you will learn to manipulate image brightness and contrast — two fundamental operations in image processing.

**What you'll do:**
- Implement brightness adjustment
- Implement contrast adjustment
- Apply optimal contrast normalization
- Use local contrast enhancement to reveal hidden details

**What you'll learn:**
- How arithmetic operations affect images
- The importance of preserving mean when adjusting contrast
- When and how to apply local vs global adjustments
- Practical techniques for enhancing dark or poorly-exposed images

**Connection to previous labs:**
- Lab 3: You saw how brightness changes broke your model
- Lab 4: You learned about color spaces
- Lab 5 (today): You learn to **control** brightness and contrast

## Setup

In [ ]:
"""
Computer Vision Course - Lab 5: Brightness and Contrast

This cell sets up the environment.
Works automatically for both local and Google Colab!
"""

import os
import sys

# Detect environment
IN_COLAB = 'google.colab' in sys.modules

print("=" * 60)
print("Computer Vision - Lab 5: Brightness and Contrast")
print("=" * 60)

if IN_COLAB:
    print("\n🔵 Running on Google Colab")
    print("-" * 60)
    
    if not os.path.exists('computer-vision'):
        print("📥 Cloning repository...")
        !git clone https://github.com/mjck/computer-vision.git
        print("✓ Repository cloned successfully")
    else:
        !git -C computer-vision pull
        print("✓ Repository updated")
    
    %cd computer-vision/labs/lab05_brightness_contrast
    print(f"✓ Current directory: {os.getcwd()}")
    
    sys.path.insert(0, '/content/computer-vision')
    print("✓ Python path configured")
    
    print("-" * 60)
    print("🟢 Colab setup complete!\n")
    
else:
    print("\n🟢 Running locally")
    print("-" * 60)
    print(f"✓ Current directory: {os.getcwd()}")
    
    repo_root = os.path.abspath('../..')
    if repo_root not in sys.path:
        sys.path.insert(0, repo_root)
    print(f"✓ Repository root: {repo_root}")
    
    print("-" * 60)
    print("🟢 Local setup complete!\n")

print("=" * 60)
print("✅ Environment ready!")
print("=" * 60)

## Import Libraries

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Import course utilities
try:
    from sdx import cv_imread, cv_imshow, cv_grayread
    print("✓ sdx module loaded")
except ImportError as e:
    print(f"❌ Could not import sdx: {e}")
    print("\nTroubleshooting:")
    print("  1. Check that sdx.py is in repository root")
    print("  2. Verify sys.path includes repository root")
    print(f"  3. Current sys.path: {sys.path[:3]}")
    raise

print("✓ All imports successful")

## Loading the Image... With a Twist

We'll use the Insper panoramic image again, but this time we'll load it as **floating-point** values instead of integers.

This has several advantages:
- No worrying about overflow when doing arithmetic
- Decimal values are preserved
- Still displays correctly with `cv_imshow()` (it handles the conversion)

In [ ]:
# Load as float (values in [0.0, 255.0] instead of [0, 255])
insper = cv_grayread('insper.png', asfloat=True)

print(f"Image shape: {insper.shape}")
print(f"Data type: {insper.dtype}")
print(f"Value range: [{insper.min():.1f}, {insper.max():.1f}]")

cv_imshow(insper)

Notice the **dots** in the array values — these indicate floating-point numbers.

Even though the values can go outside [0, 255], `cv_imshow()` automatically clips them before display.

In [ ]:
# Display the raw array to see the floats
print("Sample pixel values (top-left 5x5):")
print(insper[:5, :5])

---
## Activity 1: Brightness Adjustment

**Task:** Write a function that adjusts brightness by adding a constant `b` to all pixels.

Where:
- `b > 0` makes the image brighter
- `b < 0` makes the image darker

### Requirements

1. **Do not use loops** — use NumPy array operations
2. **Respect [0, 255] bounds** — clip values that go outside
3. Handle both positive and negative adjustments

In [ ]:
def adjust_brightness(image, b):
    """
    Adjust image brightness by adding a constant.
    
    Args:
        image: Grayscale image as numpy array (float)
        b: Brightness adjustment value (can be positive or negative)
    
    Returns:
        Adjusted image, clipped to [0, 255]
    
    TODO: Implement this function
    """
    # ── Your code here ──────────────────────────────────────────────────────
    return np.copy(image)  # Replace this line
    # ────────────────────────────────────────────────────────────────────────

print("✓ Function defined. Test it below!")

### Test Your Function

In [ ]:
# This should show a DARKER version
cv_imshow(adjust_brightness(insper, -100))
print("If the image is darker, your function works!")

In [ ]:
# This should show a BRIGHTER version
cv_imshow(adjust_brightness(insper, 100))
print("If the image is brighter, your function works!")

### 🤔 Observation

Notice what happens to bright areas when you add too much brightness — they become completely white (saturated).

Similarly, dark areas become completely black when you subtract too much.

This is called **clipping** and is inevitable when working with bounded ranges.

---
## Activity 2: Contrast Adjustment

**Task:** Write a function that adjusts contrast by multiplying pixels by a constat `a`.

**But wait!** Simply multiplying changes both contrast AND brightness. We want to change contrast while preserving the average brightness:

* Before doing so, you need to subtract the original mean of the gray levels from each pixel;

* After doing so, you need to add the original mean of the gray levels to each pixel.

Where:
- `a > 1` increases contrast (darker darks, brighter brights)
- `a < 1` decreases contrast (everything moves toward middle gray)

### Why subtract and add mean?

- Subtracting mean centers the values around zero
- Multiplying then scales the distance from the center
- Adding mean back restores the original average brightness

### Requirements

1. **Do not use loops** — use NumPy operations
2. **Respect [0, 255] bounds** — clip the result
3. **Round to nearest integer** — even though we're using floats, pixel values should be whole numbers

In [ ]:
def adjust_contrast(image, a):
    """
    Adjust image contrast by scaling around the mean.
    
    Args:
        image: Grayscale image as numpy array (float)
        a: Contrast scaling factor
           a > 1 increases contrast
           a < 1 decreases contrast
    
    Returns:
        Adjusted image, clipped to [0, 255] and rounded
    
    TODO: Implement this function
    """
    # ── Your code here ──────────────────────────────────────────────────────
    return np.copy(image)  # Replace this line
    # ────────────────────────────────────────────────────────────────────────

print("✓ Function defined. Test it below!")

### Test Your Function

In [ ]:
# This should show LOWER contrast (washed out, gray-ish)
# but NOT lower brightness (average gray level stays the same)
cv_imshow(adjust_contrast(insper, 0.5))
print("Lower contrast: Details are less visible, image looks flat")

In [ ]:
# This should show HIGHER contrast (punchier, more dramatic)
# but NOT higher brightness (average gray level stays the same)
cv_imshow(adjust_contrast(insper, 1.5))
print("Higher contrast: Details are more visible, darks darker, brights brighter")

### ✏️ Reflection Question

Compare the brightness-adjusted and contrast-adjusted images:

1. What happens to the sky when you increase brightness vs increase contrast?
2. What happens to dark areas when you decrease brightness vs decrease contrast?

Write your answers as comments in the cell below:

In [ ]:
# Your answers:

# 1. Sky with brightness vs contrast:
#    ...

# 2. Dark areas with decreased brightness vs contrast:
#    ...

---
## Optimal Contrast

Manual contrast adjustment requires guessing a good scaling factor.

An automatic alternative: **optimal contrast normalization**

**Idea:** Stretch the pixel values so that:
- The darkest pixel becomes 0
- The brightest pixel becomes 255
- Everything else scales proportionally

This maximizes the use of the available dynamic range.

### Activity 3: Implement Optimal Contrast

Write a function that normalizes an image to use the full [0, 255] range.

**Hint:** Three steps:
1. Find min and max values
2. Subtract min (shifts darkest to 0)
3. Scale by 255/(max-min) (stretches brightest to 255)

In [ ]:
def optimal_contrast(image):
    """
    Normalize image to use full [0, 255] range.
    
    Maps:
        min(image) → 0
        max(image) → 255
    
    Args:
        image: Grayscale image as numpy array
    
    Returns:
        Normalized image
    
    TODO: Implement this function
    """
    # ── Your code here ──────────────────────────────────────────────────────
    return np.copy(image)  # Replace this line
    # ────────────────────────────────────────────────────────────────────────

print("✓ Function defined")

### Test Optimal Contrast

We have three versions of the Insper image:
- Original (normal exposure)
- Dark (underexposed)
- Bright (overexposed)

A correct implementation should make all three look similar after optimization.

In [ ]:
# Load the three versions
dark = cv_grayread('insper-dark.png', asfloat=True)
bright = cv_grayread('insper-bright.png', asfloat=True)

print("Original image:")
print(f"  Range: [{insper.min():.1f}, {insper.max():.1f}]")

print("\nDark image:")
print(f"  Range: [{dark.min():.1f}, {dark.max():.1f}]")

print("\nBright image:")
print(f"  Range: [{bright.min():.1f}, {bright.max():.1f}]")

In [ ]:
# Apply optimal contrast to all three
print("After optimal contrast normalization:")
print("All three should look visually similar\n")

cv_imshow(optimal_contrast(insper))
print("Original (normalized)")

cv_imshow(optimal_contrast(dark))
print("Dark (normalized)")

cv_imshow(optimal_contrast(bright))
print("Bright (normalized)")

print("\n✓ If all three look similar, your function works correctly!")

---
## Activity 4: Revealing Hidden Details

Let's apply our techniques to a challenging image.

In [ ]:
# Load a very dark image
basement = cv_grayread('basement.png', asfloat=True)

print(f"Basement image range: [{basement.min():.1f}, {basement.max():.1f}]")
print("This is a very dark image with lots of hidden detail\n")

cv_imshow(basement)
print("The basement appears almost completely black!")

### Try Optimal Contrast

Let's see what optimal contrast reveals:

In [ ]:
cv_imshow(optimal_contrast(basement))
print("Optimal contrast reveals some detail, but not everything...")

### Your Turn: Reveal More Details

The optimal contrast helps, but there's still more information hidden in the shadows.

**Task:** Use any combination of the functions you've written (`adjust_brightness`, `adjust_contrast`, `optimal_contrast`) to create an image that reveals as much detail as possible.

**Tip:** Try combining multiple operations! For example:
- Apply optimal contrast first
- Then adjust brightness or contrast
- Experiment with different values

In [ ]:
# ── Your code here ──────────────────────────────────────────────────────────
# Try to reveal as much detail as possible from the basement image

cv_imshow(basement)  # Replace this with your enhancement pipeline
# ────────────────────────────────────────────────────────────────────────────

print("Experiment with different combinations!")

---
## Local Contrast Enhancement

### The Problem with Global Adjustments

Optimal contrast on the basement image is limited because:
- Even a *single* bright pixel limits how much we can stretch
- The max value prevents us from amplifying the dark regions

**Solution:** Apply optimal contrast to **regions** instead of the whole image!

### NumPy Slices Are References

Important concept: When you slice a NumPy array, you get a **reference**, not a copy.

This means modifications to the slice affect the original array:

```python
output = image.copy()
roi = output[y1:y2, x1:x2]  # roi is a reference to part of output
roi[:] = process(roi)         # modifies output directly
```

Let's use this to enhance specific regions.

In [ ]:
# Example: Enhance the table area in the background
output = basement.copy()

# Select region of interest (ROI)
# Rows 240-310, columns 0-90
roi = output[240:310, 0:90]

# Apply optimal contrast to just this region
roi[:] = optimal_contrast(roi)

cv_imshow(output)
print("Notice how the table details in the back-left are now visible!")

### 🕵️ Challenge 3: Find the Hidden Object

There's something... *interesting* hidden in this basement.

**Task:** Use local optimal contrast to find it.

**Hints:**
- It's somewhere in the center-right area
- You'll need to try different ROI coordinates
- The syntax is: `output[y1:y2, x1:x2]`
- Remember: rows are Y (vertical), columns are X (horizontal)

**Getting the coordinates:**
1. Display the image with `cv_imshow()`
2. Hover over areas to see coordinates (in Jupyter/Colab)
3. Identify the suspicious region
4. Adjust the ROI bounds to isolate it

In [ ]:
# ── Your code here ──────────────────────────────────────────────────────────
output = basement.copy()

# Replace these coordinates to find the hidden object
roi = output[240:310, 0:90]  # Change these values!

roi[:] = optimal_contrast(roi)

cv_imshow(output)
# ────────────────────────────────────────────────────────────────────────────

print("Keep experimenting with different regions until you find it!")

### 🤔 What Did You Find?

Write a brief description of what you discovered:

In [ ]:
# What I found in the basement:
# ...

# Approximate coordinates of the hidden object:
# rows: y1 to y2
# cols: x1 to x2


---
## Summary

### What You Learned Today

1. **Brightness adjustment** 
   - Simple addition to all pixels
   - Shifts the entire histogram
   - Can cause clipping at boundaries

2. **Contrast adjustment** 
   - Scales pixel distances from mean
   - Preserves average brightness
   - Enhances or reduces differences

3. **Optimal contrast** 
   - Automatic stretch to full range
   - Good for correcting exposure
   - Limited by global min/max

4. **Local contrast enhancement**
   - Apply adjustments to regions
   - Reveals details that global methods miss
   - Uses NumPy slice references

### Key Insights

- **Global vs local:** Sometimes adjusting the whole image isn't enough
- **Clipping matters:** Saturated pixels lose information permanently
- **Mean preservation:** Critical for contrast without brightness shift
- **Dynamic range:** Using the full [0, 255] range maximizes information

### Connection to Computer Vision

These techniques are foundational for:
- **Preprocessing:** Normalize images before feeding to models
- **Data augmentation:** Make models robust to lighting variations (Lab 3!)
- **HDR imaging:** Combine multiple exposures
- **Medical imaging:** Enhance X-rays, MRIs to reveal anatomy
- **Security cameras:** Enhance dark footage

### ✏️ Final Reflection

Answer these questions:

1. Which technique (brightness, contrast, optimal, local) would you use for each scenario?
   - A photo taken in dim light:
   - A photo that looks "washed out":
   - A photo where one area is too dark:

2. Why is it important to preserve mean when adjusting contrast?

In [ ]:
# Your reflection:

# 1. Best technique for each scenario:
#    Dim light: ...
#    Washed out: ...
#    One dark area: ...

# 2. Why preserve mean for contrast:
#    ...

---
## 📋 Submission Checklist

Before submitting, make sure:

- [ ] Activity 1: `adjust_brightness()` implemented and tested
- [ ] Activity 2: `adjust_contrast()` implemented and tested
- [ ] Reflection questions answered (brightness vs contrast)
- [ ] Activity 3: `optimal_contrast()` implemented and tested
- [ ] Activity 4: Attempted to enhance basement image
- [ ] Challenge 3: Found the hidden object in the basement
- [ ] Final reflection questions answered
- [ ] All cells executed in order

**Above Average (+1 point):** Creative enhancement of the basement image